# Task 04 - Pré Processamento e Limpeza dos Dados

Sub-tarefas:
- Codificação (encoding) dos dados.
- Escalonamento dos dados.
- Regras de negócio para limpar dados.

Definição de pronto:

- Datasets de treino e testes preprocessados


Como visto na Task 01 (Análise Exploratória de Dados), todos os dados são numéricos e estão preenchidos. Logo não há necessidade de codificação de atributos categórios e preenchimento de valores faltantes. 

A respeito dos outliers e valores extremos, este problema de classificação se trata em certa instância de uma identificação de anomalias (bebês com risco de saúde). Sendo assim, valores outliers podem ajudar a identificar padrões fora do comportamento normal esperado. 

O escalonamento dos dados será feito durante a etapa de validação cruzada. Justificamos essa decisão para evitar possíveis vazamentos de dados entre os conjuntos de treino e validação dos k-folds.

Sendo assim, esta Task 04 pode ser considerada como Pronta desde já.


# Task 05 - Extração e Seleção de Atributos

Subtarefas:
- Fazer código experimental para selecionar atributos automaticamente.
- Implementar classe que seleciona os atributos automaticamente.


Definição de Pronto
- Ter um código experimental para selecionar os atributos.
- Ter uma classe implementada que a partir de um dataset de treino gera um conjunto de atributos selecionados. 
- Ter um conjunto de atributos selecionadas salvas em arquivo (artefato).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,cross_val_score

from sklearn.metrics import f1_score

from sklearn.base import BaseEstimator

from src.data_access_handler import DataAccessHandler

In [2]:
access_handler = DataAccessHandler(main_path=".")
df_train = access_handler.load(dataset_type="train")

A seleção de features será executada através do método *Recursive Feature Elimination* (Eliminação Recursiva de Atributos), que busca reduzir recursivamente o número de atributos disponíveis, porém sem perder poder preditivo. Este método utiliza as importâncias dos atributos de um modelo de machine learning treinado, para medir qual o grau das features na predição final.  Além disso, será usado validação cruzada pra atingir um maior grau de robustez na seleção.

A métrica que irá avaliar essa seleção de atributos será a F1-score, com média macro. Isto é, levará em consideração todos os verdadeiros positivos, falsos positivos, falsos negativos.

In [3]:
def f1_score_micro(estimator:BaseEstimator,X:pd.DataFrame,y:pd.DataFrame):
    y_pred = estimator.predict(X)
    return f1_score(y_true=y,y_pred=y_pred, average='micro')

rfe_feature_selector = RFECV(estimator=RandomForestClassifier(min_samples_split=3, min_samples_leaf=3,random_state=42),
                            step=1,
                            min_features_to_select=3,
                            cv = StratifiedKFold(n_splits=5,random_state=42,shuffle=True),
                            scoring=f1_score_micro,
                            verbose=2,
                            n_jobs=-1)

In [4]:
target = ['fetal_health']
features = df_train.drop(columns=target).columns
X,y = df_train[features],df_train[target].values.ravel()

rfe_feature_selector.fit(X,y)

Fitting estimator with 21 features.
Fitting estimator with 21 features.
Fitting estimator with 21 features.
Fitting estimator with 21 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 20 features.
Fitting estimator with 20 features.
Fitting estimator with 20 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 19 features.
Fitting estimator with 19 features.
Fitting estimator with 19 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 18 features.
Fitting estimator with 18 features.
Fitting estimator with 18 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 17 features.
Fitting estimator with 17 features.
Fitting estimator with 17 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 16 features.
Fitting estimator with 16 fe

RFECV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
      estimator=RandomForestClassifier(min_samples_leaf=3, min_samples_split=3,
                                       random_state=42),
      min_features_to_select=3, n_jobs=-1,
      scoring=<function f1_score_micro at 0x29ea90160>, verbose=2)

In [5]:
selected_features = rfe_feature_selector.get_feature_names_out()
X,y = df_train[selected_features],df_train[target].values.ravel()

scores = cross_val_score(estimator=RandomForestClassifier(min_samples_split=3, min_samples_leaf=3,random_state=42),
                cv = StratifiedKFold(n_splits=5,random_state=42,shuffle=True),
                X=X,y=y,
                scoring=f1_score_micro,
                n_jobs=-1)

print("Atributos Selecionados:",selected_features)
print("\nF1-score Final da Validação Cruzada",scores.mean())

Atributos Selecionados: ['baseline value' 'accelerations' 'uterine_contractions'
 'prolongued_decelerations' 'abnormal_short_term_variability'
 'mean_value_of_short_term_variability'
 'percentage_of_time_with_abnormal_long_term_variability'
 'mean_value_of_long_term_variability' 'histogram_width' 'histogram_min'
 'histogram_mode' 'histogram_mean' 'histogram_median']

F1-score Final da Validação Cruzada 0.9306666666666666


Utilizando o código experimental implementado em uma classe separada FeatureSelector

In [1]:
from feature_selector import FeatureSelector
from data_access_handler import DataAccessHandler

access_handler = DataAccessHandler(main_path=".")
df_train = access_handler.load(dataset_type="train")

target = 'fetal_health'
X,y = df_train.drop(columns=target),df_train[target].values.ravel()
selector = FeatureSelector()
selector.select_best_features(X=X,y=y)

Atributos Selecionados: ['baseline value', 'accelerations', 'uterine_contractions', 'prolongued_decelerations', 'abnormal_short_term_variability', 'mean_value_of_short_term_variability', 'percentage_of_time_with_abnormal_long_term_variability', 'mean_value_of_long_term_variability', 'histogram_width', 'histogram_min', 'histogram_mode', 'histogram_mean', 'histogram_median']


In [3]:
selector.save_best_features(path = ".")

In [1]:

selector = FeatureSelector()
selector.load_best_features(path="./")
selector.get_selected_features

['baseline value',
 'accelerations',
 'uterine_contractions',
 'prolongued_decelerations',
 'abnormal_short_term_variability',
 'mean_value_of_short_term_variability',
 'percentage_of_time_with_abnormal_long_term_variability',
 'mean_value_of_long_term_variability',
 'histogram_width',
 'histogram_min',
 'histogram_mode',
 'histogram_mean',
 'histogram_median']

# Conclusões

- 13 atributos finais foram selecionados utilizando o método RFECV, modelo RandomForest e mantendo a seed aleatória com o valor de 42 (para reproductibilidade).
- A classe que fara essa seleção automática está implementada em um arquivo separado "feature_selector.py"
- A classe que seleciona os atributos salva eles em um arquivo yaml, para ser lido em tempo de inferêcia/testes.